# Libraries

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline

In [31]:
import datetime as dt

In [ ]:
# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
# import contextily as ctx
# pd.set_option('display.max_columns', None)  
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('max_colwidth', -1)
# import tilemapbase
# import folium

In [3]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import os
from IPython.display import display
import json

# Dataset import and preparation

In [4]:
aebm_cascadia = pd.read_parquet("./AEBM_CasadiaResults.parquet")
# Convert from dataframe to geodataframe
aebm_cascadia_gdf = gpd.GeoDataFrame(
    aebm_cascadia, geometry=gpd.points_from_xy(
        aebm_cascadia.Longitude, aebm_cascadia.Latitude))
aebm_cascadia_gdf.crs = {'init': 'epsg:4326'}

In [5]:
# Create new columns for building class, type, design level
aebm_cascadia_gdf['occupancy_class'] = aebm_cascadia_gdf['ProfileNam'].str[:4]
aebm_cascadia_gdf['building_type'] = aebm_cascadia_gdf['ProfileNam'].str[4:6]
aebm_cascadia_gdf['DesignLevel'] = aebm_cascadia_gdf['ProfileNam'].str[6:8]

In [6]:
aebm_cascadia_gdf['loss_ratio'] = (aebm_cascadia_gdf.LossStruct 
                                   + aebm_cascadia_gdf.LossNonStr
                                   + aebm_cascadia_gdf.LossNonS_1)/aebm_cascadia_gdf.BldgValue

In [7]:
null = None

In [8]:
aebm_cascadia_gdf['OccAgg'] = aebm_cascadia_gdf['occupancy_class'].apply(
    lambda occ: str(occ)[:3])

In [9]:
aebm_cascadia_gdf['ComLossRatio'] = aebm_cascadia_gdf.apply(
    lambda row: row.loss_ratio if row.OccAgg == 'COM' else null, axis=1)

In [10]:
aebm_cascadia_gdf['ResLossRatio'] = aebm_cascadia_gdf.apply(
    lambda row: row.loss_ratio if row.OccAgg == 'RES' else null, axis=1)

# round up casualties

In [11]:
def round_up(number):
    return np.ceil(number / 10.) * 10.

In [12]:
aebm_cascadia_gdf['DaytimeInjuries'] = aebm_cascadia_gdf.apply(
    lambda row: round_up(row.CasDayLvl1 + row.CasDayLvl2 + row.CasDayLvl3), axis=1)

In [13]:
aebm_cascadia_gdf['DaytimeDeaths'] = aebm_cascadia_gdf.apply(
    lambda row: round_up(row.CasDayLvl4), axis=1)

In [14]:
aebm_cascadia_gdf['NighttimeInjuries'] = aebm_cascadia_gdf.apply(
    lambda row: round_up(row.CasNight_1 + row.CasNight_2 + row.CasNight_3), axis=1)

aebm_cascadia_gdf['NighttimeDeaths'] = aebm_cascadia_gdf.apply(
    lambda row: round_up(row.CasNightLv), axis=1)

In [18]:
aebm_cascadia_gdf = aebm_cascadia_gdf.rename(
    columns={"eqAebmId": "id"})

In [15]:
save_building_loss_shp = False
save_casualties_shp = False

In [17]:
building_loss_cols = [
        'id', 
        'geometry',  # this is needed
        'ComLossRatio', 
        'ResLossRatio'
       ]

loss_ratio_gdf = aebm_cascadia_gdf.rename.copy()[building_loss_cols]
if save_building_loss_shp:
    output_file = "building_loss_ratio.json"
    print(f'Saving file to {output_file}')
    loss_ratio_gdf.to_file(output_file, drive="GeoJSON")

In [20]:
casualties_cols = ['id', 
        'geometry',  # this is needed
        'DaytimeInjuries', 
        'DaytimeDeaths', 
        'NighttimeInjuries', 
        'NighttimeDeaths'
       ]
casualties_gdf = aebm_cascadia_gdf.copy()[casualties_cols]
if save_casualties_shp:
    output_file = "casualties.json"
    print(f'Saving file to {output_file}')
    casualties_gdf.to_file(output_file, drive="GeoJSON")

# create helper function

In [19]:
def create_feature_collection_row(row, ix, property_cols):
    feature_dict = {}
    feature_dict["id"] = ix + 1
    feature_dict["type"] = "Feature"
    feature_dict["geometry"] = {
        "type": "Point", 
        "coordinates": [row.Longitude, row.Latitude]
    }
    feature_dict["properties"] = json.loads(
        row[property_cols].to_json())
    
    return feature_dict

# output data with only building loss ratio

In [28]:
property_cols = ['ComLossRatio',
 'DaytimeDeaths',
 'DaytimeInjuries',
 'NighttimeDeaths',
 'NighttimeInjuries',
 'ResLossRatio',
 #'geometry',
 'id']

In [32]:
today_date = dt.datetime.now().strftime('%Y-%m-%d')

In [33]:
json_cols = [ 
 'Longitude', 
 'Latitude'] + property_cols


feature_collections = [
    create_feature_collection_row(
        aebm_cascadia_gdf[json_cols].iloc[ix], ix,
        property_cols
    )
    for ix in aebm_cascadia_gdf.index
]
json_body = {
    "count": 27371,
    "next": "http://service.civicpdx.org/disaster-resilience/api/AebmResults/",
    "previous": None,
    "results": {
        "type": "FeatureCollection",
        "features": feature_collections}
}
output_file = f"aebm_loss_casualties_{today_date}.json"
with open(output_file, "w") as f:
    print(f"saving file to {output_file}")
    json.dump(json_body, f)

saving file to aebm_loss_casualties_2019-09-30.json


# output data with only RES loss ratio

In [ ]:
res_property_cols = [
 'RES_loss_ratio',
]
res_json_cols = [ 
 'Longitude', 
 'Latitude'] + res_property_cols


res_feature_collections = [
    create_feature_collection_row(
        aebm_cascadia_gdf[res_json_cols].iloc[ix], ix,
        res_property_cols
    )
    for ix in aebm_cascadia_gdf.index
]
res_json_body = {
    "count": 27371,
    "next": "http://service.civicpdx.org/disaster-resilience/api/AebmResults/",
    "previous": None,
    "results": {
        "type": "FeatureCollection",
        "features": res_feature_collections}
}
with open("aebm_res_loss_ratio.json", "w") as f:
    json.dump(res_json_body, f)

# output data as plain json

In [ ]:
property_cols = [
 'id',
 'COM_loss_ratio',
 'RES_loss_ratio',
 'Daytime_injury',
 'Daytime_deaths',
 'Nighttime_injury',
 'Nighttime_deaths'
]

json_cols = ['eqAebmId',
 'geometry',
 'COM_loss_ratio',
 'RES_loss_ratio',
 'Daytime_injury',
 'Daytime_deaths',
 'Nighttime_injury',
 'Nighttime_deaths', 
 'Longitude', 
 'Latitude']

feature_collections = [
    create_feature_collection_row(
        aebm_cascadia_gdf[json_cols].iloc[ix], ix,
        property_cols
    )
    for ix in aebm_cascadia_gdf.index
]
json_body = {
    "count": 27371,
    "next": "http://service.civicpdx.org/disaster-resilience/api/AebmResults/",
    "previous": None,
    "results": {
        "type": "FeatureCollection",
        "features": feature_collections}
}
with open("aebm_story_cards_plain_json.json", "w") as f:
    json.dump(json_body, f)